In [116]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [117]:
import warnings

warnings.filterwarnings("ignore")

In [118]:
df = pd.read_csv("train.csv", encoding="utf-8")

In [119]:
df

,Cluster/Country/Region,SubCluster,calweek,brand,dc,product_code,customer_code,Quantity
0,US,SC1,202001,BRANDA,DC1,PRDCT1,CUSTA,537
1,US,SC1,202002,BRANDA,DC1,PRDCT1,CUSTA,544
2,US,SC1,202003,BRANDA,DC1,PRDCT1,CUSTA,574
3,US,SC1,202004,BRANDA,DC1,PRDCT1,CUSTA,523
4,US,SC1,202005,BRANDA,DC1,PRDCT1,CUSTA,571
...,...,...,...,...,...,...,...,...
18463,US,SC2,202311,BRANDC,DC4,PRDCT9,CUSTZ,1753
18464,US,SC2,202312,BRANDC,DC4,PRDCT9,CUSTZ,1752
18465,US,SC2,202313,BRANDC,DC4,PRDCT9,CUSTZ,1673
18466,US,SC2,202314,BRANDC,DC4,PRDCT9,CUSTZ,1752


In [120]:
df.columns

Index(['Cluster/Country/Region', 'SubCluster', 'calweek', 'brand', 'dc',
       'product_code', 'customer_code', 'Quantity'],
      dtype='object')

In [121]:
df['timeseries_key'] = df['product_code'] + '_' + df['customer_code']

In [122]:
df

,Cluster/Country/Region,SubCluster,calweek,brand,dc,product_code,customer_code,Quantity,timeseries_key
0,US,SC1,202001,BRANDA,DC1,PRDCT1,CUSTA,537,PRDCT1_CUSTA
1,US,SC1,202002,BRANDA,DC1,PRDCT1,CUSTA,544,PRDCT1_CUSTA
2,US,SC1,202003,BRANDA,DC1,PRDCT1,CUSTA,574,PRDCT1_CUSTA
3,US,SC1,202004,BRANDA,DC1,PRDCT1,CUSTA,523,PRDCT1_CUSTA
4,US,SC1,202005,BRANDA,DC1,PRDCT1,CUSTA,571,PRDCT1_CUSTA
...,...,...,...,...,...,...,...,...,...
18463,US,SC2,202311,BRANDC,DC4,PRDCT9,CUSTZ,1753,PRDCT9_CUSTZ
18464,US,SC2,202312,BRANDC,DC4,PRDCT9,CUSTZ,1752,PRDCT9_CUSTZ
18465,US,SC2,202313,BRANDC,DC4,PRDCT9,CUSTZ,1673,PRDCT9_CUSTZ
18466,US,SC2,202314,BRANDC,DC4,PRDCT9,CUSTZ,1752,PRDCT9_CUSTZ


In [141]:
df['timeseries_key'].value_counts()

timeseries_key
PRDCT1_CUSTA    171
PRDCT3_CUSTM    171
PRDCT5_CUSTL    171
PRDCT5_CUSTK    171
PRDCT4_CUSTZ    171
               ... 
PRDCT6_CUSTC    171
PRDCT6_CUSTB    171
PRDCT6_CUSTA    171
PRDCT5_CUSTF    171
PRDCT9_CUSTZ    171
Name: count, Length: 108, dtype: int64

In [123]:
df1 = df.drop(['Cluster/Country/Region', 'SubCluster', 'brand', 'dc'], axis = 1)

In [124]:
df1

,calweek,product_code,customer_code,Quantity,timeseries_key
0,202001,PRDCT1,CUSTA,537,PRDCT1_CUSTA
1,202002,PRDCT1,CUSTA,544,PRDCT1_CUSTA
2,202003,PRDCT1,CUSTA,574,PRDCT1_CUSTA
3,202004,PRDCT1,CUSTA,523,PRDCT1_CUSTA
4,202005,PRDCT1,CUSTA,571,PRDCT1_CUSTA
...,...,...,...,...,...
18463,202311,PRDCT9,CUSTZ,1753,PRDCT9_CUSTZ
18464,202312,PRDCT9,CUSTZ,1752,PRDCT9_CUSTZ
18465,202313,PRDCT9,CUSTZ,1673,PRDCT9_CUSTZ
18466,202314,PRDCT9,CUSTZ,1752,PRDCT9_CUSTZ


In [125]:
df1['timeseries_key'] = df['product_code'] + '_' + df['customer_code']

In [126]:
df1['timeseries_key'].unique()

array(['PRDCT1_CUSTA', 'PRDCT1_CUSTB', 'PRDCT1_CUSTC', 'PRDCT1_CUSTD',
       'PRDCT1_CUSTE', 'PRDCT1_CUSTF', 'PRDCT2_CUSTA', 'PRDCT2_CUSTB',
       'PRDCT2_CUSTC', 'PRDCT2_CUSTD', 'PRDCT2_CUSTE', 'PRDCT2_CUSTF',
       'PRDCT3_CUSTA', 'PRDCT3_CUSTB', 'PRDCT3_CUSTC', 'PRDCT3_CUSTD',
       'PRDCT3_CUSTE', 'PRDCT3_CUSTF', 'PRDCT4_CUSTA', 'PRDCT4_CUSTB',
       'PRDCT4_CUSTC', 'PRDCT4_CUSTD', 'PRDCT4_CUSTE', 'PRDCT4_CUSTF',
       'PRDCT5_CUSTA', 'PRDCT5_CUSTB', 'PRDCT5_CUSTC', 'PRDCT5_CUSTD',
       'PRDCT5_CUSTE', 'PRDCT5_CUSTF', 'PRDCT6_CUSTA', 'PRDCT6_CUSTB',
       'PRDCT6_CUSTC', 'PRDCT6_CUSTD', 'PRDCT6_CUSTE', 'PRDCT6_CUSTF',
       'PRDCT7_CUSTA', 'PRDCT7_CUSTB', 'PRDCT7_CUSTC', 'PRDCT7_CUSTD',
       'PRDCT7_CUSTE', 'PRDCT7_CUSTF', 'PRDCT8_CUSTA', 'PRDCT8_CUSTB',
       'PRDCT8_CUSTC', 'PRDCT8_CUSTD', 'PRDCT8_CUSTE', 'PRDCT8_CUSTF',
       'PRDCT9_CUSTA', 'PRDCT9_CUSTB', 'PRDCT9_CUSTC', 'PRDCT9_CUSTD',
       'PRDCT9_CUSTE', 'PRDCT9_CUSTF', 'PRDCT1_CUSTK', 'PRDCT1_CUSTL',
      

In [127]:
timeseries_key_values= ['PRDCT1_CUSTA', 'PRDCT1_CUSTB', 'PRDCT1_CUSTC', 'PRDCT1_CUSTD',
       'PRDCT1_CUSTE', 'PRDCT1_CUSTF', 'PRDCT2_CUSTA', 'PRDCT2_CUSTB',
       'PRDCT2_CUSTC', 'PRDCT2_CUSTD', 'PRDCT2_CUSTE', 'PRDCT2_CUSTF',
       'PRDCT3_CUSTA', 'PRDCT3_CUSTB', 'PRDCT3_CUSTC', 'PRDCT3_CUSTD',
       'PRDCT3_CUSTE', 'PRDCT3_CUSTF', 'PRDCT4_CUSTA', 'PRDCT4_CUSTB',
       'PRDCT4_CUSTC', 'PRDCT4_CUSTD', 'PRDCT4_CUSTE', 'PRDCT4_CUSTF',
       'PRDCT5_CUSTA', 'PRDCT5_CUSTB', 'PRDCT5_CUSTC', 'PRDCT5_CUSTD',
       'PRDCT5_CUSTE', 'PRDCT5_CUSTF', 'PRDCT6_CUSTA', 'PRDCT6_CUSTB',
       'PRDCT6_CUSTC', 'PRDCT6_CUSTD', 'PRDCT6_CUSTE', 'PRDCT6_CUSTF',
       'PRDCT7_CUSTA', 'PRDCT7_CUSTB', 'PRDCT7_CUSTC', 'PRDCT7_CUSTD',
       'PRDCT7_CUSTE', 'PRDCT7_CUSTF', 'PRDCT8_CUSTA', 'PRDCT8_CUSTB',
       'PRDCT8_CUSTC', 'PRDCT8_CUSTD', 'PRDCT8_CUSTE', 'PRDCT8_CUSTF',
       'PRDCT9_CUSTA', 'PRDCT9_CUSTB', 'PRDCT9_CUSTC', 'PRDCT9_CUSTD',
       'PRDCT9_CUSTE', 'PRDCT9_CUSTF', 'PRDCT1_CUSTK', 'PRDCT1_CUSTL',
       'PRDCT1_CUSTM', 'PRDCT1_CUSTX', 'PRDCT1_CUSTY', 'PRDCT1_CUSTZ',
       'PRDCT2_CUSTK', 'PRDCT2_CUSTL', 'PRDCT2_CUSTM', 'PRDCT2_CUSTX',
       'PRDCT2_CUSTY', 'PRDCT2_CUSTZ', 'PRDCT3_CUSTK', 'PRDCT3_CUSTL',
       'PRDCT3_CUSTM', 'PRDCT3_CUSTX', 'PRDCT3_CUSTY', 'PRDCT3_CUSTZ',
       'PRDCT4_CUSTK', 'PRDCT4_CUSTL', 'PRDCT4_CUSTM', 'PRDCT4_CUSTX',
       'PRDCT4_CUSTY', 'PRDCT4_CUSTZ', 'PRDCT5_CUSTK', 'PRDCT5_CUSTL',
       'PRDCT5_CUSTM', 'PRDCT5_CUSTX', 'PRDCT5_CUSTY', 'PRDCT5_CUSTZ',
       'PRDCT6_CUSTK', 'PRDCT6_CUSTL', 'PRDCT6_CUSTM', 'PRDCT6_CUSTX',
       'PRDCT6_CUSTY', 'PRDCT6_CUSTZ', 'PRDCT7_CUSTK', 'PRDCT7_CUSTL',
       'PRDCT7_CUSTM', 'PRDCT7_CUSTX', 'PRDCT7_CUSTY', 'PRDCT7_CUSTZ',
       'PRDCT8_CUSTK', 'PRDCT8_CUSTL', 'PRDCT8_CUSTM', 'PRDCT8_CUSTX',
       'PRDCT8_CUSTY', 'PRDCT8_CUSTZ', 'PRDCT9_CUSTK', 'PRDCT9_CUSTL',
       'PRDCT9_CUSTM', 'PRDCT9_CUSTX', 'PRDCT9_CUSTY', 'PRDCT9_CUSTZ']

In [128]:
# ARIMA model
def arima_forecast(train, test):
    model = ARIMA(train, order=(1, 0, 0))  # ARIMA(p, d, q)
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=len(test))
    return forecast

# SARIMA model
def sarima_forecast(train, test):
    model = SARIMAX(train, order=(1, 0, 0), seasonal_order=(1, 0, 0, 12))  # SARIMA(p, d, q)(P, D, Q, m)
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=len(test))
    return forecast

# Holt-Winters model
def holt_winters_forecast(train, test):
    model = ExponentialSmoothing(train, seasonal_periods=12, trend='add', seasonal='add')
    model_fit = model.fit()
    forecast = model_fit.forecast(len(test))
    return forecast

# Facebook Prophet model
def prophet_forecast(train, test):
    model = Prophet()
    model.fit(train)
    future = model.make_future_dataframe(periods=len(test), freq='W')
    forecast = model.predict(future)
    forecast = forecast['yhat'][-len(test):].values
    return forecast
# Create an empty dataframe to store predicted values and MAPE
# results_df = pd.DataFrame(columns=['date', 'TimeSeries_Key' ,'Prophet_Predicted', 'Prophet_MAPE', 'ARIMA_MAPE', 'SARIMA_MAPE',
#                                    'HoltWinters_MAPE'])
results_df = pd.DataFrame()

In [129]:
  for i in timeseries_key_values:
    df_prod1 = df1[df1['timeseries_key']==i]
    
    print("----------RESULT FOR",i,"----------")
    
    # Convert 'calweek' column to datetime format
    df_prod1['calweek'] = pd.to_datetime(df_prod1['calweek'].astype(str) + '1', format='%Y%W%w')
    
    # Group data by 'calweek' and aggregate 'Quantity'
    ts_data = df_prod1.groupby('calweek')['Quantity'].sum().reset_index()
    
    # Rename columns for Prophet model
    ts_data.rename(columns={'calweek': 'ds', 'Quantity': 'y'}, inplace=True)

    # Split data into training and testing sets
    n_train = int(len(ts_data) * 0.8)  # 80% for training, 20% for testing
    train = ts_data.iloc[:n_train]
    test = ts_data.iloc[n_train:]
    
    # Perform forecasting using ARIMA
    arima_predictions = arima_forecast(train['y'], test['y'])

    # Perform forecasting using SARIMA
    sarima_predictions = sarima_forecast(train['y'], test['y'])

    # Perform forecasting using Holt-Winters
    holt_winters_predictions = holt_winters_forecast(train['y'], test['y'])

    # Perform forecasting using Facebook Prophet
    prophet_predictions = prophet_forecast(train, test)

    # Calculate MAPE
    def calculate_mape(actual, forecast):
        return np.mean(np.abs((actual - forecast) / actual)) * 100

    # Calculate MAPE for ARIMA
    arima_mape = calculate_mape(test['y'], arima_predictions)

    # Calculate MAPE for SARIMA
    sarima_mape = calculate_mape(test['y'], sarima_predictions)

    # Calculate MAPE for Holt-Winters
    holt_winters_mape = calculate_mape(test['y'], holt_winters_predictions)

    # Calculate MAPE for Facebook Prophet
    prophet_mape = calculate_mape(test['y'], prophet_predictions)

    # Print MAPE values
    print("ARIMA - MAPE: {:.2f}%".format(arima_mape))
    print("SARIMA - MAPE: {:.2f}%".format(sarima_mape))
    print("Holt-Winters - MAPE: {:.2f}%".format(holt_winters_mape))
    print("Facebook Prophet - MAPE: {:.2f}%".format(prophet_mape))
    
    #FORECASTINGGG
    
    # Split data into training and testing sets
    train = ts_data.iloc[:172]
    
    # Facebook Prophet model
    def prophet_forecast1(train):
        model = Prophet()
        model.fit(train)
        future = model.make_future_dataframe(periods=13, freq='W')
        forecast = model.predict(future)
        forecast = forecast['yhat'][-13:].values
        return forecast
    
    # Perform forecasting using Facebook Prophet
    prophet_predictions = prophet_forecast1(train)
    
    print(prophet_predictions)
    
    # Generate date values for the forecasted period
    start_date = ts_data['ds'].iloc[-1]
    forecast_dates = pd.date_range(start=start_date, periods=len(test), freq='W').strftime('%Y-%m-%d')
    
    for j in range(13):
        results_df = pd.concat([results_df, pd.DataFrame({
            'date': [forecast_dates[j]],
            'timeseries_key': [i],
            'Prophet_Predicted': [prophet_predictions[j]],
            'Prophet_MAPE': [prophet_mape],
            'ARIMA_MAPE': [arima_mape],
            'SARIMA_MAPE': [sarima_mape],
            'HoltWinters_MAPE': [holt_winters_mape]
        })], ignore_index=True)        

----------RESULT FOR PRDCT1_CUSTA ----------


11:49:41 - cmdstanpy - INFO - Chain [1] start processing
11:49:41 - cmdstanpy - INFO - Chain [1] done processing


ARIMA - MAPE: 2.96%
SARIMA - MAPE: 3.25%
Holt-Winters - MAPE: 3.11%
Facebook Prophet - MAPE: 3.04%


11:49:42 - cmdstanpy - INFO - Chain [1] start processing
11:49:42 - cmdstanpy - INFO - Chain [1] done processing


[539.39404224 539.37677859 537.07936554 533.3934533  530.91916377
 531.7785003  535.69256105 539.91814149 541.07310235 537.48968334
 530.5287839  524.01917468 521.98616939]
----------RESULT FOR PRDCT1_CUSTB ----------


...

[1706.52467761 1712.00086985 1719.01749632 1717.99885372 1707.35258679
 1696.00262998 1694.84503086 1706.56303714 1723.98729168 1737.17856786
 1740.88036796 1735.82676211 1725.46331825]
----------RESULT FOR PRDCT9_CUSTZ ----------


11:53:08 - cmdstanpy - INFO - Chain [1] start processing
11:53:08 - cmdstanpy - INFO - Chain [1] done processing


ARIMA - MAPE: 2.05%
SARIMA - MAPE: 2.21%
Holt-Winters - MAPE: 1.98%
Facebook Prophet - MAPE: 2.10%


11:53:09 - cmdstanpy - INFO - Chain [1] start processing
11:53:09 - cmdstanpy - INFO - Chain [1] done processing


[1694.40516777 1697.31311822 1701.38783033 1693.85784694 1675.79069743
 1662.8192573  1669.18149768 1692.72428492 1716.51543811 1724.32745238
 1713.83074731 1696.0173188  1684.22505292]


In [130]:
results_df

,date,timeseries_key,Prophet_Predicted,Prophet_MAPE,ARIMA_MAPE,SARIMA_MAPE,HoltWinters_MAPE
0,2023-04-16,PRDCT1_CUSTA,539.394042,3.042337,2.961178,3.254019,3.107791
1,2023-04-23,PRDCT1_CUSTA,539.376779,3.042337,2.961178,3.254019,3.107791
2,2023-04-30,PRDCT1_CUSTA,537.079366,3.042337,2.961178,3.254019,3.107791
3,2023-05-07,PRDCT1_CUSTA,533.393453,3.042337,2.961178,3.254019,3.107791
4,2023-05-14,PRDCT1_CUSTA,530.919164,3.042337,2.961178,3.254019,3.107791
...,...,...,...,...,...,...,...
1399,2023-06-11,PRDCT9_CUSTZ,1716.515438,2.102834,2.046700,2.210724,1.979310
1400,2023-06-18,PRDCT9_CUSTZ,1724.327452,2.102834,2.046700,2.210724,1.979310
1401,2023-06-25,PRDCT9_CUSTZ,1713.830747,2.102834,2.046700,2.210724,1.979310
1402,2023-07-02,PRDCT9_CUSTZ,1696.017319,2.102834,2.046700,2.210724,1.979310


In [131]:
# Convert the date column to datetime format
results_df['date'] = pd.to_datetime(results_df['date'])

# Convert the date format to 'YYYY-WW'
results_df['calweek'] = results_df['date'].dt.strftime('%Y-%U')


In [132]:
results_df

,date,timeseries_key,Prophet_Predicted,Prophet_MAPE,ARIMA_MAPE,SARIMA_MAPE,HoltWinters_MAPE,calweek
0,2023-04-16,PRDCT1_CUSTA,539.394042,3.042337,2.961178,3.254019,3.107791,2023-16
1,2023-04-23,PRDCT1_CUSTA,539.376779,3.042337,2.961178,3.254019,3.107791,2023-17
2,2023-04-30,PRDCT1_CUSTA,537.079366,3.042337,2.961178,3.254019,3.107791,2023-18
3,2023-05-07,PRDCT1_CUSTA,533.393453,3.042337,2.961178,3.254019,3.107791,2023-19
4,2023-05-14,PRDCT1_CUSTA,530.919164,3.042337,2.961178,3.254019,3.107791,2023-20
...,...,...,...,...,...,...,...,...
1399,2023-06-11,PRDCT9_CUSTZ,1716.515438,2.102834,2.046700,2.210724,1.979310,2023-24
1400,2023-06-18,PRDCT9_CUSTZ,1724.327452,2.102834,2.046700,2.210724,1.979310,2023-25
1401,2023-06-25,PRDCT9_CUSTZ,1713.830747,2.102834,2.046700,2.210724,1.979310,2023-26
1402,2023-07-02,PRDCT9_CUSTZ,1696.017319,2.102834,2.046700,2.210724,1.979310,2023-27


In [142]:
results_df["timeseries_key"].value_counts()

timeseries_key
PRDCT1_CUSTA    13
PRDCT3_CUSTM    13
PRDCT5_CUSTL    13
PRDCT5_CUSTK    13
PRDCT4_CUSTZ    13
                ..
PRDCT6_CUSTC    13
PRDCT6_CUSTB    13
PRDCT6_CUSTA    13
PRDCT5_CUSTF    13
PRDCT9_CUSTZ    13
Name: count, Length: 108, dtype: int64

In [133]:
# # Save results_df as a CSV file
# results_df.to_csv('predictions.csv', index=False)

In [134]:
df1

,calweek,product_code,customer_code,Quantity,timeseries_key
0,202001,PRDCT1,CUSTA,537,PRDCT1_CUSTA
1,202002,PRDCT1,CUSTA,544,PRDCT1_CUSTA
2,202003,PRDCT1,CUSTA,574,PRDCT1_CUSTA
3,202004,PRDCT1,CUSTA,523,PRDCT1_CUSTA
4,202005,PRDCT1,CUSTA,571,PRDCT1_CUSTA
...,...,...,...,...,...
18463,202311,PRDCT9,CUSTZ,1753,PRDCT9_CUSTZ
18464,202312,PRDCT9,CUSTZ,1752,PRDCT9_CUSTZ
18465,202313,PRDCT9,CUSTZ,1673,PRDCT9_CUSTZ
18466,202314,PRDCT9,CUSTZ,1752,PRDCT9_CUSTZ


In [139]:
concatenated_df = pd.concat([df, results_df], ignore_index=True)
concatenated_df

,Cluster/Country/Region,SubCluster,calweek,brand,dc,product_code,customer_code,Quantity,timeseries_key,date,Prophet_Predicted,Prophet_MAPE,ARIMA_MAPE,SARIMA_MAPE,HoltWinters_MAPE
0,US,SC1,202001,BRANDA,DC1,PRDCT1,CUSTA,537.0,PRDCT1_CUSTA,NaT,NaN,NaN,NaN,NaN,NaN
1,US,SC1,202002,BRANDA,DC1,PRDCT1,CUSTA,544.0,PRDCT1_CUSTA,NaT,NaN,NaN,NaN,NaN,NaN
2,US,SC1,202003,BRANDA,DC1,PRDCT1,CUSTA,574.0,PRDCT1_CUSTA,NaT,NaN,NaN,NaN,NaN,NaN
3,US,SC1,202004,BRANDA,DC1,PRDCT1,CUSTA,523.0,PRDCT1_CUSTA,NaT,NaN,NaN,NaN,NaN,NaN
4,US,SC1,202005,BRANDA,DC1,PRDCT1,CUSTA,571.0,PRDCT1_CUSTA,NaT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19867,NaN,NaN,2023-24,NaN,NaN,NaN,NaN,NaN,PRDCT9_CUSTZ,2023-06-11,1716.515438,2.102834,2.0467,2.210724,1.97931
19868,NaN,NaN,2023-25,NaN,NaN,NaN,NaN,NaN,PRDCT9_CUSTZ,2023-06-18,1724.327452,2.102834,2.0467,2.210724,1.97931
19869,NaN,NaN,2023-26,NaN,NaN,NaN,NaN,NaN,PRDCT9_CUSTZ,2023-06-25,1713.830747,2.102834,2.0467,2.210724,1.97931
19870,NaN,NaN,2023-27,NaN,NaN,NaN,NaN,NaN,PRDCT9_CUSTZ,2023-07-02,1696.017319,2.102834,2.0467,2.210724,1.97931


In [136]:
concatenated_df[18468:]

,calweek,product_code,customer_code,Quantity,timeseries_key,date,Prophet_Predicted,Prophet_MAPE,ARIMA_MAPE,SARIMA_MAPE,HoltWinters_MAPE
18468,2023-16,NaN,NaN,NaN,PRDCT1_CUSTA,2023-04-16,539.394042,3.042337,2.961178,3.254019,3.107791
18469,2023-17,NaN,NaN,NaN,PRDCT1_CUSTA,2023-04-23,539.376779,3.042337,2.961178,3.254019,3.107791
18470,2023-18,NaN,NaN,NaN,PRDCT1_CUSTA,2023-04-30,537.079366,3.042337,2.961178,3.254019,3.107791
18471,2023-19,NaN,NaN,NaN,PRDCT1_CUSTA,2023-05-07,533.393453,3.042337,2.961178,3.254019,3.107791
18472,2023-20,NaN,NaN,NaN,PRDCT1_CUSTA,2023-05-14,530.919164,3.042337,2.961178,3.254019,3.107791
...,...,...,...,...,...,...,...,...,...,...,...
19867,2023-24,NaN,NaN,NaN,PRDCT9_CUSTZ,2023-06-11,1716.515438,2.102834,2.046700,2.210724,1.979310
19868,2023-25,NaN,NaN,NaN,PRDCT9_CUSTZ,2023-06-18,1724.327452,2.102834,2.046700,2.210724,1.979310
19869,2023-26,NaN,NaN,NaN,PRDCT9_CUSTZ,2023-06-25,1713.830747,2.102834,2.046700,2.210724,1.979310
19870,2023-27,NaN,NaN,NaN,PRDCT9_CUSTZ,2023-07-02,1696.017319,2.102834,2.046700,2.210724,1.979310


In [137]:
# Merge the dataframes based on 'TimeSeries_Key'
merged_df = pd.merge(results_df, df, on='timeseries_key', how='inner')
merged_df

,date,timeseries_key,Prophet_Predicted,Prophet_MAPE,ARIMA_MAPE,SARIMA_MAPE,HoltWinters_MAPE,calweek_x,Cluster/Country/Region,SubCluster,calweek_y,brand,dc,product_code,customer_code,Quantity
0,2023-04-16,PRDCT1_CUSTA,539.394042,3.042337,2.961178,3.254019,3.107791,2023-16,US,SC1,202001,BRANDA,DC1,PRDCT1,CUSTA,537
1,2023-04-16,PRDCT1_CUSTA,539.394042,3.042337,2.961178,3.254019,3.107791,2023-16,US,SC1,202002,BRANDA,DC1,PRDCT1,CUSTA,544
2,2023-04-16,PRDCT1_CUSTA,539.394042,3.042337,2.961178,3.254019,3.107791,2023-16,US,SC1,202003,BRANDA,DC1,PRDCT1,CUSTA,574
3,2023-04-16,PRDCT1_CUSTA,539.394042,3.042337,2.961178,3.254019,3.107791,2023-16,US,SC1,202004,BRANDA,DC1,PRDCT1,CUSTA,523
4,2023-04-16,PRDCT1_CUSTA,539.394042,3.042337,2.961178,3.254019,3.107791,2023-16,US,SC1,202005,BRANDA,DC1,PRDCT1,CUSTA,571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240079,2023-07-09,PRDCT9_CUSTZ,1684.225053,2.102834,2.046700,2.210724,1.979310,2023-28,US,SC2,202311,BRANDC,DC4,PRDCT9,CUSTZ,1753
240080,2023-07-09,PRDCT9_CUSTZ,1684.225053,2.102834,2.046700,2.210724,1.979310,2023-28,US,SC2,202312,BRANDC,DC4,PRDCT9,CUSTZ,1752
240081,2023-07-09,PRDCT9_CUSTZ,1684.225053,2.102834,2.046700,2.210724,1.979310,2023-28,US,SC2,202313,BRANDC,DC4,PRDCT9,CUSTZ,1673
240082,2023-07-09,PRDCT9_CUSTZ,1684.225053,2.102834,2.046700,2.210724,1.979310,2023-28,US,SC2,202314,BRANDC,DC4,PRDCT9,CUSTZ,1752


In [138]:
# Select columns from df2 to add to df1
columns_to_add = df[["timeseries_key", "Cluster/Country/Region", "SubCluster", "brand", "dc"]]

# Join the selected columns from df2 to df1
merged = results_df.join(columns_to_add.set_index('timeseries_key'), on='timeseries_key')
merged

,date,timeseries_key,Prophet_Predicted,Prophet_MAPE,ARIMA_MAPE,SARIMA_MAPE,HoltWinters_MAPE,calweek,Cluster/Country/Region,SubCluster,brand,dc
0,2023-04-16,PRDCT1_CUSTA,539.394042,3.042337,2.961178,3.254019,3.107791,2023-16,US,SC1,BRANDA,DC1
0,2023-04-16,PRDCT1_CUSTA,539.394042,3.042337,2.961178,3.254019,3.107791,2023-16,US,SC1,BRANDA,DC1
0,2023-04-16,PRDCT1_CUSTA,539.394042,3.042337,2.961178,3.254019,3.107791,2023-16,US,SC1,BRANDA,DC1
0,2023-04-16,PRDCT1_CUSTA,539.394042,3.042337,2.961178,3.254019,3.107791,2023-16,US,SC1,BRANDA,DC1
0,2023-04-16,PRDCT1_CUSTA,539.394042,3.042337,2.961178,3.254019,3.107791,2023-16,US,SC1,BRANDA,DC1
...,...,...,...,...,...,...,...,...,...,...,...,...
1403,2023-07-09,PRDCT9_CUSTZ,1684.225053,2.102834,2.046700,2.210724,1.979310,2023-28,US,SC2,BRANDC,DC4
1403,2023-07-09,PRDCT9_CUSTZ,1684.225053,2.102834,2.046700,2.210724,1.979310,2023-28,US,SC2,BRANDC,DC4
1403,2023-07-09,PRDCT9_CUSTZ,1684.225053,2.102834,2.046700,2.210724,1.979310,2023-28,US,SC2,BRANDC,DC4
1403,2023-07-09,PRDCT9_CUSTZ,1684.225053,2.102834,2.046700,2.210724,1.979310,2023-28,US,SC2,BRANDC,DC4


In [143]:
merged["timeseries_key"].value_counts()

timeseries_key
PRDCT1_CUSTA    2223
PRDCT3_CUSTM    2223
PRDCT5_CUSTL    2223
PRDCT5_CUSTK    2223
PRDCT4_CUSTZ    2223
                ... 
PRDCT6_CUSTC    2223
PRDCT6_CUSTB    2223
PRDCT6_CUSTA    2223
PRDCT5_CUSTF    2223
PRDCT9_CUSTZ    2223
Name: count, Length: 108, dtype: int64